In [3]:
# -*- coding: utf-8 -*-
"""
================================================================================
| SCRIPT DE OTIMIZAÇÃO DO SEQUENCIAMENTO DE PINTURA - NATHOR                   |
================================================================================
|                                                                              |
| Versão: 3.0                         |
| Autor: Manuel Finda                                                            |
| Data: 2025-09-03                                                             |
|                                                                              |
|------------------------------------------------------------------------------|
| DESCRIÇÃO GERAL                                                              |
|------------------------------------------------------------------------------|
| Este script constitui uma Prova de Conceito (POC) avançada para a otimização |
| do sequenciamento de produção na área de pintura da Nathor. O sistema        |
| utiliza um algoritmo híbrido (heurística + meta-heurística) para gerar um    |
| cronograma de produção diário que busca o equilíbrio ótimo entre duas metas  |
| principais:                                                                  |
|                                                                              |
| 1. RESPEITO ÀS PRIORIDADES DE NEGÓCIO: Garantir que as ordens de produção     |
|    mais urgentes, com base no saldo de estoque e datas de entrega, sejam     |
|    produzidas primeiro.                                                      |
|                                                                              |
| 2. MÁXIMA EFICIÊNCIA OPERACIONAL: Minimizar o tempo de máquina parada         |
|    (setups), reduzindo o número de trocas de cor e de tipo de peça.          |
|                                                                              |
| O algoritmo considera múltiplas restrições do "chão de fábrica", como a      |
| capacidade de fornecimento da metalurgia, o limite de armazenamento em       |
| "gaiolas" e a disponibilidade de "gancheiras".                               |
|                                                                              |
|------------------------------------------------------------------------------|
| FLUXO DE EXECUÇÃO DO SCRIPT                                                  |
|------------------------------------------------------------------------------|
| 1. Pré-processamento: Regras de negócio, como a antecipação de peças         |
|    especiais, são aplicadas aos dados de entrada.                            |
| 2. Estágio 1 (Planejamento): O algoritmo principal aloca as ordens de        |
|    produção nos dias, respeitando todas as restrições de capacidade.        |
| 3. Estágio 2 (Otimização): Para cada dia planejado, a sequência de produção  |
|    é refinada pela meta-heurística Busca Tabu para minimizar os custos de    |
|    setup.                                                                    |
| 4. Exportação: O resultado final (cronograma otimizado e lista de itens     |
|    rejeitados) é salvo em um arquivo Excel (.xlsx) para fácil análise.       |
|                                                                              |
|------------------------------------------------------------------------------|
| DEPENDÊNCIAS                                                                 |
|------------------------------------------------------------------------------|
| - pandas: Para manipulação de dados e exportação para Excel.                 |
| - openpyxl: Motor para a escrita de arquivos .xlsx pelo pandas.              |
|   (Instalação: pip install pandas openpyxl)                                  |
|                                                                              |
================================================================================
"""

# --- 1. BIBLIOTECAS ---
# Importação das bibliotecas necessárias para a execução do script.
import pandas as pd
from datetime import datetime, timedelta
import copy
import time
import math
import random

# --- 2. CONFIGURAÇÃO E PARÂMETROS GLOBAIS ---
# Nesta seção, definimos as constantes que governam as regras de negócio
# e as capacidades da linha de produção. Alterar estes valores impacta
# diretamente o resultado da simulação.

# Custos de tempo para setups (em minutos), baseados no feedback da Nathor.
SETUP_TIME_COLOR = 15
SETUP_TIME_PIECE = 3

# Capacidade produtiva diária total (em minutos).
# Calculado com base no horário de funcionamento: 05h10 às 23h45.
DAILY_CAPACITY_MINUTES = 18 * 60 + 35

# Parâmetros físicos da linha de produção.
# Origem: Reunião de alinhamento 2 (Sprint 4).
MONOVIA_LENGHT_METERS = 168

# --- 3. DADOS DE ENTRADA (Simulados a partir do PDF "Dados Essenciais") ---
# Em um ambiente de produção real, estes dados seriam carregados de fontes
# externas (planilhas, APIs, banco de dados). Para esta simulação, eles
# são definidos estaticamente aqui, representando a "fotografia" do estado
# da produção em um determinado momento.

# REGRA DE NEGÓCIO: Lista de peças que necessitam de antecipação.
# Componentes que passam por processos adicionais (ex: injeção) após a pintura.
PECAS_COM_PROCESSO_ADICIONAL = ['206010040003.0'] # Exemplo: PÉ DE VELA
DIAS_ANTECIPACAO = 2

# RESTRIÇÃO DE CAPACIDADE: Fornecimento diário da Metalurgia.
# Representa a quantidade máxima de cada peça que a metalurgia consegue fornecer
# por dia para a pintura.
FORNECIMENTO_METALURGIA = {
    '206010180003.0': 1500, '208010180003.0': 1200,
    '204010180002.0': 1000, '206010040003.0': 800
}

# RESTRIÇÃO DE CAPACIDADE: Armazenamento Pós-Pintura (Gaiolas).
# Representa a quantidade máxima de cada peça pintada que pode ser armazenada
# simultaneamente.
CAPACIDADE_GAIOLAS = {
    '206010180003.0': 2000, '208010180003.0': 1000,
    '204010180002.0': 3000, '206010040003.0': 1500
}

# DADOS PRINCIPAIS: Ordens de Produção a serem planejadas.
# Geração de uma massa de dados para simular um cenário com múltiplas ordens.
print("Gerando massa de dados de produção para simulação...")
ordens_de_producao = []
product_codes = list(FORNECIMENTO_METALURGIA.keys())
cores = ['PRETO', 'AZUL', 'BRANCO', 'VERMELHO', 'AMARELO', 'VERDE']
start_date = datetime.now()

for _ in range(50):
    code = random.choice(product_codes)
    ordem = {
        'CODIGO_PRODUTO': code, 'Tinta': random.choice(cores),
        'Quantidade_Planejada': random.randint(300, 1600),
        'Estoque': random.randint(0, 500),
        'Data_de_Entrega': start_date + timedelta(days=random.randint(2, 10)),
        'Tempo_Calculado': round(random.uniform(0.5, 1.5), 4),
        'Pecas_por_Gancheira': random.randint(2, 8),
        'ESTOQUE_GANCHEIRA': random.randint(100, 250),
        'DISTANCIA_M': round(random.uniform(0.5, 1.2), 2)
    }
    ordem['Pedidos'] = ordem['Quantidade_Planejada']
    ordens_de_producao.append(ordem)
print(f"{len(ordens_de_producao)} ordens de produção simuladas foram geradas.")

# --- 4. FUNÇÕES DE LÓGICA E OTIMIZAÇÃO ---

def preprocessar_pedidos(lista_de_pedidos):
    """Aplica regras de negócio aos pedidos antes do início do planejamento.

    Esta função atua como uma camada de pré-processamento, "limpando" e
    ajustando os dados de entrada para que o algoritmo principal possa operar
    de forma mais simples e direta. A principal regra implementada aqui é a
    antecipação da data de entrega para peças que possuem processos
    subsequentes à pintura, tornando-as artificialmente mais urgentes.

    Args:
        lista_de_pedidos (list): A lista original de dicionários, onde cada
                                 dicionário representa uma ordem de produção.

    Returns:
        list: A lista de pedidos com as datas de entrega ajustadas conforme
              as regras de negócio. Retorna uma cópia profunda para não
              modificar a lista original.
    """
    pedidos_ajustados = copy.deepcopy(lista_de_pedidos)
    for item in pedidos_ajustados:
        if item['CODIGO_PRODUTO'] in PECAS_COM_PROCESSO_ADICIONAL:
            item['Data_de_Entrega'] -= timedelta(days=DIAS_ANTECIPACAO)
            item['Observacao'] = f'Entrega antecipada em {DIAS_ANTECIPACAO} dias (proc. adicional)'
    return pedidos_ajustados

def calculate_cost(sequence):
    """Calcula o custo TOTAL de setup de uma determinada sequência de produção.

    Esta é a função objetivo para o otimizador do Estágio 2. Ela quantifica
    a eficiência de uma sequência medindo o tempo total perdido em setups.
    A meta do otimizador é encontrar a permutação da sequência que minimiza o
    valor retornado por esta função.

    Args:
        sequence (list): Uma lista de dicionários, representando uma sequência
                         de produção para um dia.

    Returns:
        int: O custo total de setup em minutos, que é a soma de todas as
             trocas de cor (15 min) e de peça (3 min) na sequência.
    """
    total_setup_cost = 0
    if len(sequence) < 2:
        return 0
    # Itera por todos os pares adjacentes de itens na sequência
    for i in range(len(sequence) - 1):
        # Adiciona custo se a cor mudar
        if sequence[i]['Tinta'] != sequence[i+1]['Tinta']:
            total_setup_cost += SETUP_TIME_COLOR
        # Adiciona custo se o tipo de peça mudar
        if sequence[i]['CODIGO_PRODUTO'] != sequence[i+1]['CODIGO_PRODUTO']:
            total_setup_cost += SETUP_TIME_PIECE
    return total_setup_cost

def calculate_prioritization_score(item):
    """Calcula uma pontuação de prioridade para um item de produção.

    Esta função é o núcleo da lógica de negócio do Estágio 1. Ela converte
    as regras de prioridade da Nathor em uma pontuação matemática que permite
    uma ordenação inequívoca de todas as ordens de produção. A ordenação é
    feita em três níveis hierárquicos para desempate.

    Args:
        item (dict): O dicionário representando um item de produção.

    Returns:
        tuple: Uma tupla de 3 elementos. O Python ordena tuplas elemento por
               elemento, o que implementa a hierarquia de prioridades:
               1. Urgência (0 para urgente, 1 para normal)
               2. Data de Entrega (em formato timestamp, menor é mais prioritário)
               3. Tamanho do Déficit (negativo para priorizar o maior déficit)
    """
    saldo = item['Estoque'] - item['Pedidos']
    due_date = item['Data_de_Entrega']
    
    # Nível 1: Urgência. Itens com saldo negativo são sempre prioritários.
    prioridade_urgente = 1 if saldo >= 0 else 0
    # Nível 2: Data de Entrega. Desempata pela data mais próxima.
    prioridade_data = due_date.timestamp()
    # Nível 3: Tamanho do Déficit. Desempata pela maior necessidade de produção.
    prioridade_saldo = abs(saldo) if saldo < 0 else 0
    
    return (prioridade_urgente, prioridade_data, -prioridade_saldo)

def daily_sequencing_and_scheduling(unscheduled_items):
    """ESTÁGIO 1: Planeja a produção diária, aplicando todas as restrições.

    Esta é a função mais complexa do sistema, responsável por simular o processo
    de planejamento diário. Ela decide QUAIS itens serão produzidos em CADA dia.

    O processo interno pode ser resumido em:
    1. Filtra ordens que são permanentemente inviáveis (ex: falta de gancheiras).
    2. Entra em um loop que simula cada dia de produção.
    3. A cada dia, reseta os recursos diários (metalurgia, gaiolas).
    4. Ordena todos os itens restantes pela prioridade de negócio.
    5. Itera pela lista ordenada e tenta "encaixar" cada item no dia corrente,
       passando por uma série de filtros de capacidade (metalurgia, gaiolas, tempo).
    6. Itens que passam em todos os filtros são agendados para o dia. Os que
       falham são guardados para serem avaliados no dia seguinte.
    7. O loop termina quando não há mais itens para planejar ou quando os
       itens restantes não cabem em nenhum dia.

    Args:
        unscheduled_items (list): A lista completa de itens a serem planejados,
                                  já pré-processada.

    Returns:
        tuple: Uma tupla contendo duas listas:
               - schedule (list): O cronograma de produção, onde cada elemento
                 é um dicionário representando um dia.
               - permanently_rejected (list): Lista de itens que não puderam
                 ser planejados, com o motivo da rejeição.
    """
    schedule = []
    permanently_rejected = []
    plannable_items = []

    # Passo 1: Filtro de inviabilidade permanente. Remove itens que NUNCA
    # poderiam ser planejados.
    for item in unscheduled_items:
        gancheiras_necessarias = math.ceil(item['Quantidade_Planejada'] / item['Pecas_por_Gancheira'])
        comprimento_na_monovia = gancheiras_necessarias * item['DISTANCIA_M']
        
        # Lógica condicional de gancheiras, conforme reunião.
        if comprimento_na_monovia < MONOVIA_LENGHT_METERS and gancheiras_necessarias > item['ESTOQUE_GANCHEIRA']:
            item['Motivo_Rejeicao'] = f"Gancheiras insuficientes ({gancheiras_necessarias} > {item['ESTOQUE_GANCHEIRA']})"
            permanently_rejected.append(item)
        else:
            plannable_items.append(item) # Item é considerado planejável

    # Passo 2: Loop de planejamento diário sobre os itens planejáveis.
    day_number = 1
    while plannable_items:
        # A cada novo dia, reseta os contadores de consumo de recursos diários.
        consumo_metalurgia_diario = {code: 0 for code in FORNECIMENTO_METALURGIA}
        consumo_gaiolas_diario = {code: 0 for code in CAPACIDADE_GAIOLAS}
        
        plannable_items.sort(key=calculate_prioritization_score)
        
        items_for_today = []
        items_for_next_days = [] # Itens que não cabem hoje, mas podem caber amanhã.
        time_used_today, last_color, last_piece_code = 0, None, None

        # Itera sobre a lista de itens prioritários para tentar preencher o dia.
        for item in plannable_items:
            cod_produto = item['CODIGO_PRODUTO']
            qtd_planejada = item['Quantidade_Planejada']
            passes_daily_checks = True

            # Filtro 2: Fornecimento da Metalurgia
            if consumo_metalurgia_diario.get(cod_produto, 0) + qtd_planejada > FORNECIMENTO_METALURGIA.get(cod_produto, float('inf')):
                passes_daily_checks = False
            
            # Filtro 3: Capacidade de Armazenamento (Gaiolas)
            if consumo_gaiolas_diario.get(cod_produto, 0) + qtd_planejada > CAPACIDADE_GAIOLAS.get(cod_produto, float('inf')):
                passes_daily_checks = False
            
            # Filtro 4: Capacidade de Tempo
            item_time = item['Tempo_Calculado'] * 60
            setup_cost = 0
            if items_for_today:
                if last_color != item['Tinta']: setup_cost += SETUP_TIME_COLOR
                if last_piece_code != cod_produto: setup_cost += SETUP_TIME_PIECE
            
            if time_used_today + item_time + setup_cost > DAILY_CAPACITY_MINUTES:
                passes_daily_checks = False
            
            if passes_daily_checks:
                # Se o item passou em todos os filtros, é agendado para o dia.
                items_for_today.append(item)
                # E os recursos do dia são consumidos.
                time_used_today += item_time + setup_cost
                consumo_metalurgia_diario[cod_produto] += qtd_planejada
                consumo_gaiolas_diario[cod_produto] += qtd_planejada
                last_color, last_piece_code = item['Tinta'], cod_produto
            else:
                # Se falhou em algum filtro diário, vai para a lista do dia seguinte.
                items_for_next_days.append(item)
        
        if not items_for_today:
            # Se não foi possível agendar NENHUM item hoje, os restantes são
            # considerados não planejáveis por falta de capacidade.
            for item in items_for_next_days:
                item['Motivo_Rejeicao'] = "Não coube no cronograma (falta de capacidade diária)"
                permanently_rejected.append(item)
            break # Encerra o planejamento.
            
        # Adiciona o dia preenchido ao cronograma final.
        schedule.append({'day': day_number, 'items': items_for_today})
        # A lista para o próximo dia de planejamento é a lista de itens que não couberam hoje.
        plannable_items = items_for_next_days
        day_number += 1
        
    return schedule, permanently_rejected

def tabu_search_optimizer(daily_sequence, tabu_tenure=7, max_iterations=100):
    """ESTÁGIO 2: Otimiza a sequência de um único dia usando Busca Tabu."""
    # ... (A documentação desta função da resposta anterior já está completa)
    current_solution = list(daily_sequence)
    best_solution = list(daily_sequence)
    best_cost = calculate_cost(best_solution)
    tabu_list = []

    for _ in range(max_iterations):
        best_neighbor, best_neighbor_cost, best_move = None, float('inf'), None
        for i in range(len(current_solution)):
            for j in range(i + 1, len(current_solution)):
                neighbor = list(current_solution)
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                move = tuple(sorted((i, j)))
                if move in tabu_list: continue
                
                neighbor_cost = calculate_cost(neighbor)
                if neighbor_cost < best_neighbor_cost:
                    best_neighbor, best_neighbor_cost, best_move = neighbor, neighbor_cost, move

        if best_neighbor:
            current_solution = best_neighbor
            tabu_list.append(best_move)
            if len(tabu_list) > tabu_tenure: tabu_list.pop(0)
            if best_neighbor_cost < best_cost: best_solution, best_cost = best_neighbor, best_neighbor_cost
    
    return best_solution

def export_to_excel(schedule, rejected_items, filename="cronograma_otimizado_sprint4_documentado.xlsx"):
    """Exporta o resultado final para um arquivo Excel com duas abas."""
    # ... (A documentação desta função da resposta anterior já está completa)
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        # Aba 1: Cronograma Otimizado
        processed_data = []
        for day_data in schedule:
            for order, item in enumerate(day_data['items'], 1):
                saldo = item['Estoque'] - item['Pedidos']
                row = {'Dia': day_data['day'], 'Ordem_no_Dia': order, 'Saldo': saldo, **item}
                processed_data.append(row)
        
        if processed_data:
            df_schedule = pd.DataFrame(processed_data)
            df_schedule['Data_de_Entrega'] = df_schedule['Data_de_Entrega'].dt.strftime('%Y-%m-%d')
            column_order = [
                'Dia', 'Ordem_no_Dia', 'CODIGO_PRODUTO', 'Tinta', 'Quantidade_Planejada',
                'Estoque', 'Pedidos', 'Saldo', 'Data_de_Entrega', 'Tempo_Calculado',
                'Pecas_por_Gancheira', 'ESTOQUE_GANCHEIRA', 'DISTANCIA_M', 'Observacao'
            ]
            df_schedule = df_schedule.reindex(columns=column_order)
            df_schedule.to_excel(writer, sheet_name='Cronograma_Otimizado', index=False)
            print(f"\nAba 'Cronograma_Otimizado' gerada com sucesso.")

        # Aba 2: Itens Não Planejados
        if rejected_items:
            df_rejected = pd.DataFrame(rejected_items)
            if 'Data_de_Entrega' in df_rejected.columns:
                df_rejected['Data_de_Entrega'] = df_rejected['Data_de_Entrega'].dt.strftime('%Y-%m-%d')
            
            rejected_cols = [
                'CODIGO_PRODUTO', 'Tinta', 'Quantidade_Planejada',
                'Pedidos', 'Data_de_Entrega', 'Motivo_Rejeicao'
            ]
            df_rejected = df_rejected.reindex(columns=rejected_cols)
            df_rejected.to_excel(writer, sheet_name='Itens_Nao_Planejados', index=False)
            print(f"Aba 'Itens_Nao_Planejados' gerada com sucesso.")

    print(f"\nArquivo '{filename}' salvo com sucesso.")


# --- 5. EXECUÇÃO PRINCIPAL DO SCRIPT ---
if __name__ == "__main__":
    # O bloco __main__ é o ponto de entrada do script. Ele orquestra a chamada
    # das funções na ordem correta para gerar o resultado final.
    start_time = time.time()
    
    print("\nPASSO 1: Aplicando regras de negócio (pré-processamento)...")
    pedidos_prontos = preprocessar_pedidos(ordens_de_producao)
    
    print("PASSO 2: Executando Estágio 1 (Planejamento e alocação diária)...")
    initial_schedule, rejected_items = daily_sequencing_and_scheduling(pedidos_prontos)
    
    print("PASSO 3: Executando Estágio 2 (Otimização da sequência de cada dia)...")
    optimized_schedule = []
    for day_data in initial_schedule:
        refined_sequence = tabu_search_optimizer(day_data['items'])
        optimized_schedule.append({'day': day_data['day'], 'items': refined_sequence})
    print("Otimização concluída.")
    
    print("PASSO 4: Exportando resultados para o arquivo Excel...")
    export_to_excel(optimized_schedule, rejected_items)

    end_time = time.time()
    print(f"\nProcesso finalizado com sucesso em {end_time - start_time:.2f} segundos.")

Gerando massa de dados de produção para simulação...
50 ordens de produção simuladas foram geradas.

PASSO 1: Aplicando regras de negócio (pré-processamento)...
PASSO 2: Executando Estágio 1 (Planejamento e alocação diária)...
PASSO 3: Executando Estágio 2 (Otimização da sequência de cada dia)...
Otimização concluída.
PASSO 4: Exportando resultados para o arquivo Excel...

Aba 'Cronograma_Otimizado' gerada com sucesso.
Aba 'Itens_Nao_Planejados' gerada com sucesso.

Arquivo 'cronograma_otimizado_sprint4_documentado.xlsx' salvo com sucesso.

Processo finalizado com sucesso em 0.09 segundos.
